<a href="https://colab.research.google.com/github/gulizhasan/ImageClassification/blob/main/AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load the CIFAR-10 dataset
(x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Split the full training set into training and validation sets (80% training, 20% validation)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)

# Normalize the images
x_train = x_train.astype('float32') / 255
x_val = x_val.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# One-hot encode the labels
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Data Augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)

# Print the shape of the datasets to confirm
print("Training set shape (features):", x_train.shape)
print("Training set shape (labels):", y_train.shape)
print("Validation set shape (features):", x_val.shape)
print("Validation set shape (labels):", y_val.shape)
print("Test set shape (features):", x_test.shape)
print("Test set shape (labels):", y_test.shape)


Training set shape (features): (40000, 32, 32, 3)
Training set shape (labels): (40000, 10)
Validation set shape (features): (10000, 32, 32, 3)
Validation set shape (labels): (10000, 10)
Test set shape (features): (10000, 32, 32, 3)
Test set shape (labels): (10000, 10)


In [ ]:
# k-NN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Reshape data for k-NN
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_val_flat = x_val.reshape(x_val.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Initialize k-NN classifier
knn = KNeighborsClassifier(n_neighbors=3)

# Train the classifier
knn.fit(x_train_flat[:10000], y_train[:10000])

# Predict on the validation set
y_val_pred = knn.predict(x_val_flat)
val_accuracy = accuracy_score(y_val, y_val_pred)
print('k-NN Validation accuracy:', val_accuracy)


k-NN Validation accuracy: 0.2142


In [9]:
# CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define a simple CNN model
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_cnn.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))


Epoch 1/10
 607/1250 [=============>................] - ETA: 27s - loss: 1.6619 - accuracy: 0.3968

KeyboardInterrupt: 

In [ ]:
# KNN
from tensorflow.keras.layers import SimpleRNN

# Reshape data for RNN
x_train_rnn = x_train.reshape(x_train.shape[0], 32*32, 3)
x_val_rnn = x_val.reshape(x_val.shape[0], 32*32, 3)

# Define a simple RNN model
model_rnn = Sequential([
    SimpleRNN(50, input_shape=(32*32, 3), return_sequences=True),
    SimpleRNN(50),
    Dense(10, activation='softmax')
])

# Compile the model
model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_rnn.fit(x_train_rnn, y_train, epochs=10, validation_data=(x_val_rnn, y_val))


Epoch 1/10
1250/1250 [==============================] - 579s 461ms/step - loss: 2.2553 - accuracy: 0.1523 - val_loss: 2.2632 - val_accuracy: 0.1368
Epoch 2/10
1250/1250 [==============================] - 583s 467ms/step - loss: 2.2115 - accuracy: 0.1725 - val_loss: 2.2482 - val_accuracy: 0.1409
Epoch 3/10
1250/1250 [==============================] - 578s 462ms/step - loss: 2.2370 - accuracy: 0.1563 - val_loss: 2.3380 - val_accuracy: 0.0838
Epoch 4/10
1250/1250 [==============================] - 570s 456ms/step - loss: 2.2483 - accuracy: 0.1511 - val_loss: 2.2020 - val_accuracy: 0.1797
Epoch 5/10
1250/1250 [==============================] - 590s 472ms/step - loss: 2.1639 - accuracy: 0.1913 - val_loss: 2.1530 - val_accuracy: 0.2055
Epoch 6/10
1250/1250 [==============================] - 586s 469ms/step - loss: 2.1488 - accuracy: 0.2034 - val_loss: 2.1622 - val_accuracy: 0.1926
Epoch 7/10
1250/1250 [==============================] - 585s 468ms/step - loss: 2.1401 - accuracy: 0.2097 - val_

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

# Convert one-hot encoded labels back to integers
y_train_integers = np.argmax(y_train[:10000], axis=1)
y_val_integers = np.argmax(y_val, axis=1)

# Initialize SVM classifier
svm = SVC(kernel='linear')

# Train the classifier on a subset of data
svm.fit(x_train_flat[:10000], y_train_integers)

# Predict on the validation set
y_val_pred = svm.predict(x_val_flat)
val_accuracy = accuracy_score(y_val_integers, y_val_pred)
print('Validation accuracy:', val_accuracy)


Validation accuracy: 0.3209
